In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = "https://www.ss.lv/lv/animals/dogs/sell/"
req = requests.get(url) 
req.status_code 

200

In [3]:
def getColList(soup, default_columns = ('description', "url")):
    column_list = list(default_columns)
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] 
    column_list += headcolumns
    return column_list

In [5]:
soup = BeautifulSoup(req.text, 'lxml')
column_names = getColList(soup)
column_names 

['description', 'url', 'Šķirne', 'Vecums', 'Cena']

In [6]:
def getRow(row,colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [8]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows
    
apt_rows = getRowList(soup)
apt_rows[0]
getRow(apt_rows[0])

{'description': 'Nodosim mīļās, gādīgās, labās rokās mazos suņu bērniņus. \r\nMāte Austrumeiropas aitu sunsDobele un raj.',
 'url': 'https://ss.com/msg/lv/animals/dogs/labrador-retriever/dckcb.html',
 'Šķirne': 'Labradors retrivers',
 'Vecums': '2 mēn.',
 'Cena': '100  €'}

In [9]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]
row_ads = getRows(apt_rows)
dtemp = pd.DataFrame(row_ads, columns=column_names)
dtemp.head()


,description,url,Šķirne,Vecums,Cena
0,"Nodosim mīļās, gādīgās, labās rokās mazos suņu...",https://ss.com/msg/lv/animals/dogs/labrador-re...,Labradors retrivers,2 mēn.,100 €
1,Pārdodu 8 mēnešus vecu sunīti vārdā Bruno. \r\...,https://ss.com/msg/lv/animals/dogs/siberian-hu...,Sibīrijas haski,8 mēn.,80 €
2,Izgatavojam dažādas suņu būdas pēc Jūsu vēlmēm...,https://ss.com/msg/lv/animals/dogs/accessories...,-,-,200 €
3,Puisītis meklē jaunus saimniekus. Māk nokārtot...,https://ss.com/msg/lv/animals/dogs/russkiy-toy...,Krievu toiterjers (glu,2.5 mēn.,650 €
4,Ручная работа. Вязаная одежда для собачек мале...,https://ss.com/msg/lv/animals/dogs/accessories...,-,-,10 €


In [13]:
dtemp.to_csv("dtemp.csv", index=False)